In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPRegressor


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'content_rating']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

/home/carlo/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: cannot import name MLPRegressor

# Changing Genres into individual Columns

In [2]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

set(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'News', 'Reality-TV', 'Thriller', 'Western', 'Mystery', 'Short', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Game-Show', 'Sport', 'Biography'])


In [3]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [4]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4927


In [5]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes', 'gross', 'title_year'
                   , 'duration', 'facenumber_in_poster']
for column in to_replace_NaNs:
    movie_data[column] = replace_NaNs(movie_data[column])


# Changing Color to Numeric

In [6]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in movie_data.color:
    if item != 1:
        movie_data.color = movie_data.color.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [7]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [8]:
#replaces the English values to 1
movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in movie_data.language:
    if item != 1:
        movie_data.language = movie_data.language.replace(to_replace = item, value = 0)
        
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Remove columns with a low amount of 1's

In [9]:
for genre in genres_set:
    counter = 0
    for value in movie_data[genre]:
        if value == 1.0:
            counter+=1
    
weghalen = ['Game-Show', 'News', 'Reality-TV', 'Short', 'Film-Noir']

for name in weghalen:
    del movie_data[name]
    

# Chaning Content_Rating to Numeric

In [10]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)

# Trainset en testset

In [11]:
ratings = movie_data['imdb_score'].values
del movie_data['imdb_score']

X = movie_data.values
X_std = StandardScaler().fit_transform(X)

number_of_samples = len(ratings)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.75)
x_train = X_std[random_indices[:num_training_samples]]
y_train = ratings[random_indices[:num_training_samples]]


x_test = X_std[random_indices[num_training_samples:]]
y_test = ratings[random_indices[num_training_samples:]]
y_train = list(y_train)


# Analysis prediction function

In [12]:
def analyse_predictions(regr, x_test, y_test, statdict, doprint=False):
    regr.fit (x_train, y_train)

    prediction = regr.predict(x_test)
    results_list.append(prediction)

    # The mean squared error
    print("Mean squared error: %.2f"
          % np.mean((prediction - y_test) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % regr.score(x_test, y_test))

    
    
    predres = list(regr.predict(x_test))
    actual = list(y_test)
    highest = avg = error = 0
    lowest = predres[0]
    for i in range(len(predres)):
        diff = abs(predres[i] - actual[i])
        avg += diff
        error += abs(predres[i] - actual[i]) / actual[i]
        
        if doprint:
            print "predicted:", predres[i], 
            print "\tActual:", actual[i],
            print "\tDifference:", abs(predres[i] - actual[i])

        if diff > highest:
            highest = diff
        if diff < lowest:
            lowest = diff

    errper = error / len(x_test) * 100
    avgdif =  avg / len(predres)
    
    print "Number of Samples:", len(predres)
    print "Highest difference:", highest
    print "Lowest difference:", lowest
    print "average difference:", avgdif
    print "Error Percentage:", errper
    
    statdict['highest'].append(highest)
    statdict['lowest'].append(lowest)
    statdict['avgdif'].append(avgdif)
    statdict['errper'].append(errper)
    
    return statdict

# Variable set-up for result combination

In [13]:
results_list = []

statdict = {'errper': [], 'avgdif': [], 'highest': [], 'lowest': []}

# Linear Regression

In [14]:
from sklearn import linear_model
regr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 0.87
Variance score: 0.31
Number of Samples: 1232
Highest difference: 6.32301115636
Lowest difference: 0.000866318916382
average difference: 0.688542498759
Error Percentage: 12.9372047994


# Ridge Regression

In [15]:
regr = linear_model.Ridge()

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 0.87
Variance score: 0.31
Number of Samples: 1232
Highest difference: 6.32035105392
Lowest difference: 0.00156514727646
average difference: 0.688613770531
Error Percentage: 12.9382903956


# Lasso

In [16]:
regr = linear_model.Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 0.87
Variance score: 0.31
Number of Samples: 1232
Highest difference: 6.30663703204
Lowest difference: 0.000677759640084
average difference: 0.688939790889
Error Percentage: 12.9437323438


# Bayesian Ridge

In [17]:
regr = linear_model.BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 0.87
Variance score: 0.31
Number of Samples: 1232
Highest difference: 6.27513098687
Lowest difference: 0.000361503201298
average difference: 0.688908500637
Error Percentage: 12.9470735163


# Nearest Neighbor

In [18]:
regr = KNeighborsRegressor(n_neighbors=15, weights='distance')

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 0.81
Variance score: 0.36
Number of Samples: 1232
Highest difference: 5.22826008521
Lowest difference: 0.0
average difference: 0.669750525595
Error Percentage: 12.2608466752


# Decision Tree

In [19]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor()

statdict = analyse_predictions(regr, x_test, y_test, statdict)

Mean squared error: 1.29
Variance score: -0.01
Number of Samples: 1232
Highest difference: 5.9
Lowest difference: 0.0
average difference: 0.822077922078
Error Percentage: 14.9597459319


# combination

In [20]:
combo_results = []
n_algorithms = len(results_list)
for i in range(len(results_list[0])):
    tot = 0.0
    for a in range(n_algorithms):
        tot += results_list[a][i]
    combo_results.append(tot / n_algorithms)

predres = combo_results
actual = list(y_test)
highest = lowest = avg = error = 0
for i in range(len(predres)):
    diff = abs(predres[i] - actual[i])
    avg += diff
    error += abs(predres[i] - actual[i]) / actual[i]

    if False:
        print "predicted:", predres[i], 
        print "\tActual:", actual[i],
        print "\tDifference:", abs(predres[i] - actual[i])

    if diff > highest:
        highest = diff
    if diff < lowest:
        lowest = diff

errper = error / len(x_test) * 100
avgdif = avg / len(predres)
print "Number of Samples:", len(predres)
print "Highest difference:", highest
print "Lowest difference:", lowest
print "average difference:", avgdif
print "Error Percentage:", errper

statdict['highest'].append(highest)
statdict['lowest'].append(lowest)
statdict['avgdif'].append(avgdif)
statdict['errper'].append(errper)

Number of Samples: 1232
Highest difference: 6.05889838574
Lowest difference: 0
average difference: 0.643066458155
Error Percentage: 12.1397935736


In [21]:
n_groups = 7

fig, ax = plt.subplots()

index = np.arange(n_groups)
bar_width = 0.9

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, statdict['errper'], bar_width,
                 alpha=opacity,
                 color='b',
                 error_kw=error_config)

plt.xlabel('Regression Algorithm')
plt.ylabel('Error Percentages (in %)')
plt.title('Error Percentages for various regression algorithms')
plt.xticks(index + bar_width / 2, ('LinRegr', 'RidRegr', 'Lasso', 
                                   'Bayesian', 'kNN', 'DT', 'Combo'))
plt.tight_layout()
plt.show()